In [117]:
!pip install pgeocode


Requirement not upgraded as not directly required: pgeocode in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: pandas in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pgeocode)
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pgeocode)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pgeocode)
Requirement not upgraded as not directly required: python-dateutil>=2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pandas->pgeocode)
Requirement not upgraded as not directly required: pytz>=2011k in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pandas->pgeocode)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->pgeocode)
Requirement not u

In [118]:
import pgeocode
import pandas as pd
import requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [119]:
nomi = pgeocode.Nominatim('in')

In [120]:
nomi.query_postal_code("560072")

postal_code                                560072
country code                                   IN
place_name        Nagarbhavi, Nagarbhavi II Stage
state_name                              Karnataka
state_code                                     19
county_name                             Bengaluru
county_code                                   583
community_name                    Bangalore North
community_code                                NaN
latitude                                  13.2257
longitude                                 77.5431
accuracy                                        4
Name: 0, dtype: object

In [121]:
India = nomi._get_data('in')

In [122]:
india = nomi._index_postal_codes()

In [123]:
karnataka = india[india.state_name == "Karnataka"]

In [124]:
bengaluru = karnataka[karnataka.county_name == "Bengaluru"]

In [125]:
bengaluru.head()

,country code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
10165,IN,560001,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,12.9914,77.592244,3
10166,IN,560002,"Bangalore City, Bangalore Corporation Building",Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,13.2257,77.584350,4
10167,IN,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,13.2257,77.567350,4
10168,IN,560004,"Pampamahakavi Road, Mavalli, Thyagarajnagar, B...",Karnataka,19,Bengaluru,583.0,Bangalore South,NaN,12.9833,77.583300,1
10169,IN,560005,Fraser Town,Karnataka,19,Bengaluru,583.0,Bangalore North,NaN,12.9910,77.584300,1


In [126]:
bengaluru.shape

(106, 12)

In [127]:
bengaluru.reset_index(inplace = True)

In [128]:
bengaluru.columns

Index(['index', 'country code', 'postal_code', 'place_name', 'state_name',
       'state_code', 'county_name', 'county_code', 'community_name',
       'community_code', 'latitude', 'longitude', 'accuracy'],
      dtype='object')

In [129]:
bengaluru = bengaluru[['postal_code','place_name', 'community_name', 'latitude', 'longitude']]

In [130]:
bengaluru.head(10)

,postal_code,place_name,community_name,latitude,longitude
0,560001,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",Bangalore North,12.9914,77.592244
1,560002,"Bangalore City, Bangalore Corporation Building",Bangalore North,13.2257,77.584350
2,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Bangalore North,13.2257,77.567350
3,560004,"Pampamahakavi Road, Mavalli, Thyagarajnagar, B...",Bangalore South,12.9833,77.583300
4,560005,Fraser Town,Bangalore North,12.9910,77.584300
5,560006,"Training Command IAF, J.C.Nagar",Bangalore North,12.9910,77.584300
6,560007,"Agram, Air Force Hospital",Bangalore South,12.9910,77.584300
7,560008,"H.A.L II Stage H.O, Hulsur Bazaar",Bangalore North,12.9910,77.584300
8,560009,"K. G. Road, Bangalore Dist Offices Bldg",Bangalore North,12.9910,77.584300
9,560010,"Industrial Estate (Bangalore), Rajajinagar H.O...",Bangalore North,12.9604,77.567300


In [131]:
bengaluru.community_name.unique()

array(['Bangalore North', 'Bangalore South', 'Bengaluru', 'Bangalore',
       'Bellandur', 'Anekal', 'Hosakote', nan, 'Bangaloresouth',
       'Banglorenorth', 'Bg North'], dtype=object)

In [132]:
# 'Banglorenorth' ,'Bg North' to Bangalore North
# 'Bangaloresouth' to 'Bangalore South'
bengaluru.loc[bengaluru.community_name == 'Bangaloresouth', 'community_name'] = 'Bangalore South'
bengaluru.loc[bengaluru.community_name == 'Bangalorenorth', 'community_name'] = 'Bangalore North'
bengaluru.loc[bengaluru.community_name == 'Bg North', 'community_name'] = 'Bangalore North'


In [133]:
bengaluru.columns

Index(['postal_code', 'place_name', 'community_name', 'latitude', 'longitude'], dtype='object')

In [134]:

address = 'Bengaluru, KA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bengaluru are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bengaluru are 12.9791198, 77.5912997.


In [135]:
# create map of Bengaluru using latitude and longitude values
map_Bengaluru = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, place, community in zip(bengaluru['latitude'], bengaluru['longitude'], bengaluru['place_name'], bengaluru['community_name']):
    label = '{}, {}'.format(community, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bengaluru)  
    
map_Bengaluru

In [136]:
bengaluru.groupby("community_name").count()

,postal_code,place_name,latitude,longitude
community_name,,,,
Anekal,4,4,4,4
Bangalore,7,7,7,7
Bangalore North,57,57,57,57
Bangalore South,31,31,31,31
Banglorenorth,1,1,1,1
Bellandur,1,1,1,1
Bengaluru,2,2,2,2
Hosakote,2,2,2,2


In [137]:
#as Bangalore north has highest count, will be using for further clustering

In [138]:
blr_north = bengaluru[bengaluru['community_name'] == 'Bangalore North'].reset_index(drop=True)

In [139]:
blr_north.head()

,postal_code,place_name,community_name,latitude,longitude
0,560001,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",Bangalore North,12.9914,77.592244
1,560002,"Bangalore City, Bangalore Corporation Building",Bangalore North,13.2257,77.584350
2,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Bangalore North,13.2257,77.567350
3,560005,Fraser Town,Bangalore North,12.9910,77.584300
4,560006,"Training Command IAF, J.C.Nagar",Bangalore North,12.9910,77.584300


In [140]:
#Lets plot Bangalore North Map
address = 'Bangalore North,KA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore North are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore North are 12.9791198, 77.5912997.


In [141]:
# create map of Bengaluru using latitude and longitude values
map_blrNorth = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, place, community in zip(blr_north['latitude'], blr_north['longitude'], blr_north['place_name'], blr_north['community_name']):
    label = '{}, {}'.format(community, place)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_blrNorth)  
    
map_blrNorth

In [142]:
CLIENT_ID = 'LXRTQ4XM1403FUBFBJC5FV5IR4AYGWK4KNMBDC3ITQNW0XNX' # your Foursquare ID
CLIENT_SECRET = 'BRD5IUBAEQ55J3WPIJHSSP3CURLSYUURJKMA3BVSLUES0X4B' # your Foursquare Secret
VERSION = '20190131' # Foursquare API version

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius


print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LXRTQ4XM1403FUBFBJC5FV5IR4AYGWK4KNMBDC3ITQNW0XNX
CLIENT_SECRET:BRD5IUBAEQ55J3WPIJHSSP3CURLSYUURJKMA3BVSLUES0X4B


In [143]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [144]:
blrNorth_venues = getNearbyVenues(names=blr_north['place_name'],
                                   latitudes=blr_north['latitude'],
                                   longitudes=blr_north['longitude']
                                  )

Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambedkar Veedhi, Mahatma Gandhi Road, CMM Court Complex, Bangalore G.P.O., HighCourt, Vidhana Soudha, Legislators Home
Bangalore City, Bangalore Corporation Building
Malleswaram, Palace Guttahalli, Vyalikaval Extn, Swimming Pool Extn
Fraser Town
Training Command IAF, J.C.Nagar
H.A.L II Stage H.O, Hulsur Bazaar
K. G. Road, Bangalore Dist Offices Bldg
Industrial Estate (Bangalore), Rajajinagar H.O, Rajajinagar IVth Block
Science Institute
Jalahalli East, Jalahalli H.O
Jalahalli West
Ramamurthy Nagar, Doorvaninagar, Krishnarajapuram R S
Vimanapura, NAL
Seshadripuram
Gayathrinagar, Srirampuram
Yeshwanthpur Bazar, Yeswanthpura
Magadi Road
H.A. Farm, Anandnagar (Bangalore), Hebbal Kempapura
Richmond Town, CMP Centre And School, Museum Road, Bangalore Sub Foreign Post
R T Nagar H.O, P&T Col. Kavalbyrasandra
Maruthi Sevanagar
Krishnarajapuram, Devasandra
Rameshnagar, Doddanekkundi, Marathahalli Colony, Kundalahalli
Indiranagar (Bangalore)
Vijayanagar (

In [145]:
blrNorth_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Millers 46,12.991666,77.594207,Steakhouse
1,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Ujwal Bar & Restaurant,12.992280,77.594473,Indian Restaurant
2,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Fat Buddha,12.989882,77.591254,Chinese Restaurant
3,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Imperial Restaurant,12.991150,77.593837,Indian Restaurant
4,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,"Shangri-La Hotel, Bengaluru",12.992112,77.588446,Hotel
5,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Desserted,12.993039,77.589376,French Restaurant
6,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Pasta Street - Cunningham Road,12.988385,77.593891,Italian Restaurant
7,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Alliance Française,12.991232,77.596723,Concert Hall
8,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,National Gallery of Modern Arts,12.989790,77.588389,Art Museum
9,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",12.9914,77.592244,Watson's,12.994218,77.594645,Pub


In [146]:
#lets figure out how many unique venues category
print('There are {} uniques categories.'.format(len(blrNorth_venues['Venue Category'].unique())))

There are 54 uniques categories.


In [147]:
#Let us cluster based on neighborhoods
# one hot encoding
blrNorth_onehot = pd.get_dummies(blrNorth_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blrNorth_onehot['Neighborhood'] = blrNorth_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blrNorth_onehot.columns[-1]] + list(blrNorth_onehot.columns[:-1])
blrNorth_onehot = blrNorth_onehot[fixed_columns]

blrNorth_grouped = blrNorth_onehot.groupby('Neighborhood').mean().reset_index()

In [148]:
blrNorth_grouped.head(10)

,Neighborhood,ATM,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Print Shop,Pub,Rajasthani Restaurant,Restaurant,Sandwich Place,Shopping Mall,Smoke Shop,Steakhouse,Theme Park Ride / Attraction,Vegetarian / Vegan Restaurant
0,"A F Station Yelahanka, BSF Campus Yelahanka",0.333333,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0
1,Benson Town,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0
2,C.V.Raman Nagar,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,1.0,0.0,0.0,0.0
3,Domlur,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0
4,Fraser Town,0.000000,0.00000,0.000000,0.00000,0.074074,0.037037,0.00000,0.037037,0.0,...,0.0,0.037037,0.00000,0.111111,0.00000,0.00000,0.0,0.0,0.0,0.0
5,G.K.V.K.,0.333333,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.0,0.0,0.0,0.0
6,"Gayathrinagar, Srirampuram",0.000000,0.03125,0.000000,0.03125,0.000000,0.000000,0.03125,0.031250,0.0,...,0.0,0.000000,0.03125,0.000000,0.03125,0.03125,0.0,0.0,0.0,0.0
7,"H.A. Farm, Anandnagar (Bangalore), Hebbal Kemp...",0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.0,...,0.0,0.000000,0.00000,0.000000,0.00000,0.20000,0.0,0.0,0.0,0.0
8,"H.A.L II Stage H.O, Hulsur Bazaar",0.000000,0.00000,0.000000,0.00000,0.074074,0.037037,0.00000,0.037037,0.0,...,0.0,0.037037,0.00000,0.111111,0.00000,0.00000,0.0,0.0,0.0,0.0
9,H.K.P. Road,0.000000,0.00000,0.083333,0.00000,0.083333,0.000000,0.00000,0.000000,0.0,...,0.0,0.083333,0.00000,0.083333,0.00000,0.00000,0.0,0.0,0.0,0.0


In [149]:
#Top common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [150]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
blrNorth_venues_sorted = pd.DataFrame(columns=columns)
blrNorth_venues_sorted['Neighborhood'] = blrNorth_grouped['Neighborhood']

for ind in np.arange(blrNorth_grouped.shape[0]):
    blrNorth_venues_sorted.iloc[ind, 1:] = return_most_common_venues(blrNorth_grouped.iloc[ind, :], num_top_venues)

blrNorth_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"A F Station Yelahanka, BSF Campus Yelahanka",ATM,Pizza Place,Fish & Chips Shop,Clothing Store,Farm,Electronics Store,Donut Shop,Dessert Shop,Department Store,Dance Studio
1,Benson Town,Farm,Vegetarian / Vegan Restaurant,Clothing Store,Fast Food Restaurant,Electronics Store,Donut Shop,Dessert Shop,Department Store,Dance Studio,Convenience Store
2,C.V.Raman Nagar,Smoke Shop,Vegetarian / Vegan Restaurant,Chinese Restaurant,Farm,Electronics Store,Donut Shop,Dessert Shop,Department Store,Dance Studio,Convenience Store
3,Domlur,Farm,Vegetarian / Vegan Restaurant,Clothing Store,Fast Food Restaurant,Electronics Store,Donut Shop,Dessert Shop,Department Store,Dance Studio,Convenience Store
4,Fraser Town,Hotel,Indian Restaurant,Restaurant,Café,Art Gallery,Nightclub,Lounge,Italian Restaurant,Japanese Restaurant,Boutique


In [151]:
# set number of clusters
kclusters = 20

blrNorth_grouped_clustering = blrNorth_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(blrNorth_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:24] 

array([ 0,  2,  3,  2, 11,  0, 12, 10, 11,  1,  2,  4,  4,  4,  7, 11,  9,
        2,  2,  1,  6,  2,  7,  1], dtype=int32)

In [154]:
# add clustering labels
blrNorth_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

blrNorth_merged = blr_north

blrNorth_merged = blrNorth_merged.join(blrNorth_venues_sorted.set_index('Neighborhood'), on='place_name')

blrNorth_merged.head() # check the last c

,postal_code,place_name,community_name,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,560001,"Rajbhavan (Bangalore), Vasanthanagar, Dr. Ambe...",Bangalore North,12.9914,77.592244,13.0,Indian Restaurant,Coffee Shop,Hotel,Chinese Restaurant,Italian Restaurant,Vegetarian / Vegan Restaurant,Pizza Place,Fast Food Restaurant,Concert Hall,French Restaurant
1,560002,"Bangalore City, Bangalore Corporation Building",Bangalore North,13.2257,77.584350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,560003,"Malleswaram, Palace Guttahalli, Vyalikaval Ext...",Bangalore North,13.2257,77.567350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,560005,Fraser Town,Bangalore North,12.9910,77.584300,11.0,Hotel,Indian Restaurant,Restaurant,Café,Art Gallery,Nightclub,Lounge,Italian Restaurant,Japanese Restaurant,Boutique
4,560006,"Training Command IAF, J.C.Nagar",Bangalore North,12.9910,77.584300,11.0,Hotel,Indian Restaurant,Restaurant,Café,Art Gallery,Nightclub,Lounge,Italian Restaurant,Japanese Restaurant,Boutique


In [155]:
blrNorth_merged['Cluster Labels'].replace(np.NaN, 0, inplace = True)
blrNorth_merged['Cluster Labels']=blrNorth_merged['Cluster Labels'].astype('int')

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blrNorth_merged['latitude'], blrNorth_merged['longitude'], blrNorth_merged['community_name'], blrNorth_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters